In [241]:
from collections import namedtuple

import cbitstruct as bitstruct
import numpy as np
import pandas as pd

import pixie16

In [196]:
HEADER_FORMATS = {
    0: (
        "b1u14u5u4u4u4",
        ("Finish Code", "Event Length", "Header Length", "CrateID", "SlotID", "Chan#"),
    ),
    1: ("u32", ("EVTTIME_LO",)),
    2: ("u3u13u16", ("CFD trigger source bits", "CFD Fractional Time", "EVTTIME_HI")),
    3: ("b1u15u16", ("Trace Out-of-Range Flag", "Trace Length", "Event Energy")),
#     4: ("u32", ("Energy sum trailing",)),
#     5: ("u32", ("Energy sum leading",)),
#     6: ("u32", ("Energy sum gap",)),
#     7: ("u32", ("Baseline",)),
#     8: ("u32", ("QDCSum0",)),
#     9: ("u32", ("QDCSum1",)),
#     10: ("u32", ("QDCSum2",)),
#     11: ("u32", ("QDCSum3",)),
#     12: ("u32", ("QDCSum4",)),
#     13: ("u32", ("QDCSum5",)),
#     14: ("u32", ("QDCSum6",)),
#     15: ("u32", ("QDCSum7",)),
#     16: ("u32", ("Ext_TS_Lo",)),
#     17: ("p16u16", ("Ext_TS_Hi",)),
}


bit_fmt = ""
nbits = 0
field_names = ()

for i in HEADER_FORMATS.keys():
    bit_fmt += HEADER_FORMATS[i][0]
    nbits += bitstruct.calcsize(bit_fmt)
    field_names += HEADER_FORMATS[i][1]
    
field_idxs = {name: i for i, name in enumerate(field_names)}

In [242]:
HEADER_LENGTH = bitstruct.calcsize(bit_fmt) // 8


def read_list_mode_data_linear(file):
    events = []
    with open(file, "rb") as f:
        while raw_header_bits := f.read(HEADER_LENGTH):
            try:
                header_bits = bitstruct.byteswap(HEADER_LENGTH // 4 * '4', raw_header_bits)
                events.append(bitstruct.unpack(bit_fmt, header_bits))
            except TypeError:
                break
    df = pd.DataFrame.from_records(events, columns=field_names)
    df['timestamp'] = (df['EVTTIME_LO'] + df['EVTTIME_HI'] * 2 ** 32) * 10
    df['CFD_error'] = df['CFD trigger source bits'] == 7
    df['CFD_fraction'] = np.where(df['CFD_error'], 0, ((df['CFD trigger source bits'] - 1) + df['CFD Fractional Time'] / 8192) * 2)
    return df
    

In [243]:
test_file = "../tests/read/pixie16_binary_data-full.bin"

In [244]:
events = read_list_mode_data_linear(test_file)

In [272]:
HEADER_LENGTH = bitstruct.calcsize(bit_fmt) // 8


def read_list_mode_data_linear_events(file):
    events = []
    with open(file, "rb") as f:
        while raw_header_bits := f.read(HEADER_LENGTH):
            header_bits = bitstruct.byteswap(HEADER_LENGTH // 4 * '4', raw_header_bits)
            events.append(bitstruct.unpack(bit_fmt, header_bits))
    return events

In [259]:
HEADER_LENGTH = bitstruct.calcsize(bit_fmt) // 8


def read_list_mode_data_linear_events_no_append(file):
    events = 24598 * [None]
    with open(file, "rb") as f:
        for i in range(24598):
            raw_header_bits = f.read(HEADER_LENGTH)
            header_bits = bitstruct.byteswap(HEADER_LENGTH // 4 * '4', raw_header_bits)
            events[i] = bitstruct.unpack(bit_fmt, header_bits)
    return events

In [270]:
HEADER_LENGTH = bitstruct.calcsize(bit_fmt) // 8
n_events = 24598

def read_list_mode_data_at_once(file):
    events = n_events * [None]
    with open(file, "rb") as f:
        raw_bits = f.read(n_events * HEADER_LENGTH)
        bits = bitstruct.byteswap(n_events * HEADER_LENGTH // 4 * '4', raw_bits)
    total_bits_fmt = n_events * bit_fmt
    all_events_data = bitstruct.unpack(total_bits_fmt, bits)
    data = np.reshape(all_events_data, (n_events, len(field_names)))
    return all_events_data

In [273]:
%timeit read_list_mode_data_at_once(test_file)

32.8 ms ± 405 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [274]:
%timeit pixie16.read.read_list_mode_data([test_file])

2.35 s ± 18 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [275]:
%timeit read_list_mode_data_linear_events(test_file)

30.6 ms ± 487 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [276]:
%timeit read_list_mode_data_linear(test_file)

88.3 ms ± 502 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [277]:
%timeit read_list_mode_data_linear_events_no_append(test_file)

30.1 ms ± 273 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [245]:
df = pixie16.read.read_list_mode_data([test_file])
df

,channel,crate,slot,timestamp,CFD_fraction,energy,trace,CFD_error,pileup,trace_flag,...,Esum_gap,baseline,QDCSum0,QDCSum1,QDCSum2,QDCSum3,QDCSum4,QDCSum5,QDCSum6,QDCSum7
0,10,0,2,1170569551910,0.000000,1837,[],False,False,False,...,-1,-1.0,-1,-1,-1,-1,-1,-1,-1,-1
1,9,0,2,1170570473650,3.554443,3831,[],False,False,False,...,-1,-1.0,-1,-1,-1,-1,-1,-1,-1,-1
2,9,0,2,1170570647500,-0.786621,24377,[],False,False,False,...,-1,-1.0,-1,-1,-1,-1,-1,-1,-1,-1
3,9,0,2,1170570816910,1.107178,37858,[],False,False,False,...,-1,-1.0,-1,-1,-1,-1,-1,-1,-1,-1
4,10,0,2,1170570817100,0.000000,1684,[],False,False,False,...,-1,-1.0,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24593,10,0,2,1180571693780,0.649658,16785,[],False,False,False,...,-1,-1.0,-1,-1,-1,-1,-1,-1,-1,-1
24594,10,0,2,1180571922190,-1.649170,4060,[],False,False,False,...,-1,-1.0,-1,-1,-1,-1,-1,-1,-1,-1
24595,10,0,2,1180571988040,0.000000,1832,[],False,False,False,...,-1,-1.0,-1,-1,-1,-1,-1,-1,-1,-1
24596,9,0,2,1180572262810,3.884521,26102,[],False,False,False,...,-1,-1.0,-1,-1,-1,-1,-1,-1,-1,-1


In [246]:
events

,Finish Code,Event Length,Header Length,CrateID,SlotID,Chan#,EVTTIME_LO,CFD trigger source bits,CFD Fractional Time,EVTTIME_HI,Trace Out-of-Range Flag,Trace Length,Event Energy,timestamp,CFD_error,CFD_fraction
0,False,4,4,0,2,10,1092838199,7,0,27,False,0,1837,1170569551910,True,0.000000
1,False,4,4,0,2,9,1092930373,2,6367,27,False,0,3831,1170570473650,False,3.554443
2,False,4,4,0,2,9,1092947758,0,4970,27,False,0,24377,1170570647500,False,-0.786621
3,False,4,4,0,2,9,1092964699,1,4535,27,False,0,37858,1170570816910,False,1.107178
4,False,4,4,0,2,10,1092964718,7,0,27,False,0,1684,1170570817100,True,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24593,False,4,4,0,2,10,2093052386,1,2661,27,False,0,16785,1180571693780,False,0.649658
24594,False,4,4,0,2,10,2093075227,0,1437,27,False,0,4060,1180571922190,False,-1.649170
24595,False,4,4,0,2,10,2093081812,7,0,27,False,0,1832,1180571988040,True,0.000000
24596,False,4,4,0,2,9,2093109289,2,7719,27,False,0,26102,1180572262810,False,3.884521


In [236]:
events.columns

Index(['Finish Code', 'Event Length', 'Header Length', 'CrateID', 'SlotID',
       'Chan#', 'EVTTIME_LO', 'CFD trigger source bits', 'CFD Fractional Time',
       'EVTTIME_HI', 'Trace Out-of-Range Flag', 'Trace Length', 'Event Energy',
       'timestamp'],
      dtype='object')

In [232]:
events['timestamp'] = (events['EVTTIME_LO'] + events['EVTTIME_HI'] * 2 ** 32) * 10

In [164]:
pd.DataFrame.from_records(events, columns=field_names)

,Finish Code,Event Length,Header Length,CrateID,SlotID,Chan#,EVTTIME_LO,CFD trigger source bits,CFD Fractional Time,EVTTIME_HI,...,QDCSum0,QDCSum1,QDCSum2,QDCSum3,QDCSum4,QDCSum5,QDCSum6,QDCSum7,Ext_TS_Lo,Ext_TS_Hi
0,False,4,4,0,2,10,1092838199,7,0,27,...,540713,1092947758,325713947,24377,540713,1092964699,834076699,37858,540714,20846
1,False,4,4,0,2,10,1093207816,7,0,27,...,540714,1093497740,2379874331,7515,540713,1093580432,2218590235,24256,540713,33055
2,False,4,4,0,2,9,1093791674,1,4362,27,...,540713,1093891880,2144403483,18250,540714,1093891890,1101070363,6552,540713,31498
3,False,4,4,0,2,10,1094498605,7,0,27,...,540714,1094544880,3758096411,578,540714,1094615983,3758096411,249,540713,55542
4,False,4,4,0,2,9,1094768040,2,7032,27,...,540713,1094925077,2151809051,30805,540713,1094976423,1931018267,25861,540714,938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2728,False,4,4,0,2,10,2091552265,7,0,27,...,540714,2091568645,3758096411,1669,540713,2091570095,2088173595,26245,540714,49767
2729,False,4,4,0,2,9,2091936725,3,2962,27,...,540713,2091943440,569704475,29076,540714,2091943448,3758096411,964,540713,42940
2730,False,4,4,0,2,9,2092125228,3,7943,27,...,540714,2092175034,3758096411,2025,540713,2092227644,306249755,27213,540714,56371
2731,False,4,4,0,2,9,2092505798,3,1442,27,...,540713,2092651396,1026359323,12961,540714,2092651418,3758096411,1108,540714,22665


In [26]:
pd.read_hdf("../tests/read/pixie16-events.hdf")

,channel,crate,slot,timestamp,CFD_fraction,energy,trace,CFD_error,pileup,trace_flag,Esum_trailing,Esum_leading,Esum_gap,baseline
0,10,0,2,1170569551910,0.000000,1837,[],True,0,0,0,0,0,0.0
1,9,0,2,1170570473650,3.554443,3831,[],False,0,0,0,0,0,0.0
2,9,0,2,1170570647500,-0.786621,24377,[],False,0,0,0,0,0,0.0
3,9,0,2,1170570816910,1.107178,37858,[],False,0,0,0,0,0,0.0
4,10,0,2,1170570817100,0.000000,1684,[],True,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24593,10,0,2,1180571693780,0.649658,16785,[],False,0,0,0,0,0,0.0
24594,10,0,2,1180571922190,-1.649170,4060,[],False,0,0,0,0,0,0.0
24595,10,0,2,1180571988040,0.000000,1832,[],True,0,0,0,0,0,0.0
24596,9,0,2,1180572262810,3.884521,26102,[],False,0,0,0,0,0,0.0


In [233]:
events['timestamp'].head()

0    1170569551910
1    1170570473650
2    1170570647500
3    1170570816910
4    1170570817100
Name: timestamp, dtype: int64

In [148]:
f = open(test_file, "rb")




raw_bits = f.read(4)
bits = bitstruct.byteswap('4', raw_bits)

bitstruct.unpack(HEADER_FORMATS[0][0], bits)

(False, 4, 4, 0, 2, 10)

In [212]:
f = open(test_file, "rb")




raw_bits = f.read(4*4)
bits = bitstruct.byteswap(4*'4', raw_bits)

bitstruct.unpack(bit_fmt, bits)

(False, 4, 4, 0, 2, 10, 1092838199, 7, 0, 27, False, 0, 1837)

In [133]:
HEADER_FORMATS

{0: ('b1u14u5u4u4u4',
  ('Finish Code',
   'Event Length',
   'Header Length',
   'CrateID',
   'SlotID',
   'Chan#')),
 1: ('u32', ('EVTTIME_LO',)),
 2: ('u3u13u16',
  ('CFD trigger source bits', 'CFD Fractional Time', 'EVTTIME_HI')),
 3: ('b1u15u16', ('Trace Out-of-Range Flag', 'Trace Length', 'Event Energy')),
 4: ('u32', ('Energy sum trailing',)),
 5: ('u32', ('Energy sum leading',)),
 6: ('u32', ('Energy sum gap',)),
 7: ('u32', ('Baseline',)),
 8: ('u32', ('QDCSum0',)),
 9: ('u32', ('QDCSum1',)),
 10: ('u32', ('QDCSum2',)),
 11: ('u32', ('QDCSum3',)),
 12: ('u32', ('QDCSum4',)),
 13: ('u32', ('QDCSum5',)),
 14: ('u32', ('QDCSum6',)),
 15: ('u32', ('QDCSum7',)),
 16: ('u32', ('Ext_TS_Lo',)),
 17: ('p16u16', ('Ext_TS_Hi',))}

In [110]:
len(raw_bits)

17

In [ ]:
read_list_mode_data_linear()

In [ ]:
bitstruct.unpack(bit_fmt, )